### ETL del archivo en crudo `movies_dataset.parquet` de la columna `production_companies`

In [1]:
import pandas as pd
import ast
import os

In [2]:
url = "https://github.com/FranciscoHugoLezik/Movies_data/blob/main/movies_dataset.parquet?raw=true"

movies_dataset_df = pd.read_parquet(url, engine="fastparquet")

movies_dataset_df.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [3]:
production_companies_df = movies_dataset_df[['production_companies','id']].copy()

production_companies_df.iloc[0]

production_companies    [{'name': 'Pixar Animation Studios', 'id': 3}]
id                                                                 862
Name: 0, dtype: object

In [4]:
production_companies_df.rename(columns={'id': 'movie_id'}, inplace=True)

production_companies_df.iloc[0]

production_companies    [{'name': 'Pixar Animation Studios', 'id': 3}]
movie_id                                                           862
Name: 0, dtype: object

In [5]:
production_companies_df.iloc[0]['production_companies']

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

In [6]:
production_companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   production_companies  45463 non-null  object
 1   movie_id              45466 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [7]:
production_companies_df.dropna(subset='production_companies', inplace=True)

production_companies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   production_companies  45463 non-null  object
 1   movie_id              45463 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [8]:
try:
    
    production_companies_list_dict = [
        {**production_companies_dict, 'movie_id': row['movie_id']}
        for _, row in production_companies_df.iterrows()
        for production_companies_dict in ast.literal_eval(row['production_companies'])
    ]
    
except Exception as ex:
    
    print("Ha habido una excepción", type(ex))

Ha habido una excepción <class 'TypeError'>


In [10]:
production_companies_df = production_companies_df[~production_companies_df['production_companies'].isin(['True', 'False'])]

production_companies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45460 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   production_companies  45460 non-null  object
 1   movie_id              45460 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [11]:
try:
    
    production_companies_list_dict = [
        {**production_companies_dict, 'movie_id': row['movie_id']}
        for _, row in production_companies_df.iterrows()
        for production_companies_dict in ast.literal_eval(row['production_companies'])
    ]
    
except Exception as ex:
    
    print("Ha habido una excepción", type(ex))

In [13]:
modified_production_companies_df = pd.DataFrame(production_companies_list_dict)

modified_production_companies_df.iloc[0]

name        Pixar Animation Studios
id                                3
movie_id                        862
Name: 0, dtype: object

In [14]:
modified_production_companies_df.rename(columns={'name': 'company_name', 
                                                 'id': 'company_id'}, 
                                        inplace=True)

modified_production_companies_df.iloc[0]

company_name    Pixar Animation Studios
company_id                            3
movie_id                            862
Name: 0, dtype: object

In [15]:
modified_production_companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70545 entries, 0 to 70544
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  70545 non-null  object
 1   company_id    70545 non-null  int64 
 2   movie_id      70545 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [17]:
modified_production_companies_df['company_id'] = modified_production_companies_df['company_id'].astype('str')

modified_production_companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70545 entries, 0 to 70544
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  70545 non-null  object
 1   company_id    70545 non-null  object
 2   movie_id      70545 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


In [18]:
current_dir = os.getcwd()

current_dir

'c:\\Users\\franc\\Desktop\\Proyecto_Peliculas\\notebooks\\ETL\\movies_dataset'

In [19]:
base_dir = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))

base_dir

'c:\\Users\\franc\\Desktop\\Proyecto_Peliculas'

In [20]:
file_path = os.path.join(base_dir, 'data', 'ETL_data', 'movies_dataset', 'production_companies.parquet')

file_path

'c:\\Users\\franc\\Desktop\\Proyecto_Peliculas\\data\\ETL_data\\movies_dataset\\production_companies.parquet'

In [21]:
modified_production_companies_df.to_parquet(file_path)